# <font color=blue>Assignments for "Feature Engineering - Part 2"</font>

In this assignment, you are going to use a dataset related to the US education system. Please download the ([dataset](https://www.kaggle.com/noriuk/us-education-datasets-unification-project/home)) from Kaggle. You are going to use `states_all.csv` within this dataset.

To complete this assignment, submit the Github link of the Jupyter notebook file containing solutions to the questions below. You can talk to your mentor on your head or ask Slack at office time.


In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
import seaborn as sns

states = pd.read_csv("E:/user/Notebooks/data/states_all.csv")


In [4]:
fill_list = ["ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE",
             "STATE_REVENUE", "LOCAL_REVENUE", "TOTAL_EXPENDITURE",
             "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
             "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE", "GRADES_PK_G",
             "GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", "GRADES_12_G", "GRADES_1_8_G",
             "GRADES_9_12_G", "GRADES_ALL_G"]

state = states["STATE"].unique()

for state in state:
    states.loc[states["STATE"] == state, fill_list] = states.loc[states["STATE"] == state, fill_list].interpolate()

# we drop the null values after interpolation
states.dropna(inplace=True)

**(1)** Create a variable that contains the weighted average of the grades in the dataset. The number of students in the fourth grade is different from that of the eighth grade. So you will need a weighted average!

In [5]:
states["overall_score"] = (states["GRADES_4_G"]* 
                                 ((states["AVG_MATH_4_SCORE"] + states["AVG_READING_4_SCORE"]) * 0.5) + 
                                 states["GRADES_8_G"]
                                 * ((states["AVG_MATH_8_SCORE"] + 
                                     states["AVG_READING_8_SCORE"]) * 
                                    0.5))/ (states["GRADES_4_G"] + states["GRADES_8_G"])

**(2)** What is the correlation between the variable you just created and the types of expenditures? Which expenditure item has more correlation than others?

In [6]:
states[["overall_score", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

,overall_score,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
overall_score,1.000000,0.028573,0.045707,0.030917,-0.031047,-0.054539
TOTAL_EXPENDITURE,0.028573,1.000000,0.989720,0.992138,0.943475,0.920948
INSTRUCTION_EXPENDITURE,0.045707,0.989720,1.000000,0.972288,0.898873,0.877377
SUPPORT_SERVICES_EXPENDITURE,0.030917,0.992138,0.972288,1.000000,0.953311,0.904419
OTHER_EXPENDITURE,-0.031047,0.943475,0.898873,0.953311,1.000000,0.916780
CAPITAL_OUTLAY_EXPENDITURE,-0.054539,0.920948,0.877377,0.904419,0.916780,1.000000


**(3)** Now apply the Principal Components Analysis (PCA) for the four expenditure items! How much of the total variance can be explained by the first component?

In [10]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = states[["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=1)
states["pca_1"] = sklearn_pca.fit_transform(X)

print(sklearn_pca.explained_variance_ratio_)

[0.94053166]


**(4)** What is the correlation between the GPA you created and the first principal component?

In [11]:
states[["overall_score", "pca_1", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

,overall_score,pca_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
overall_score,1.000000,-0.001974,0.028573,0.045707,0.030917,-0.031047,-0.054539
pca_1,-0.001974,1.000000,0.991716,0.966589,0.987655,0.971726,0.952952
TOTAL_EXPENDITURE,0.028573,0.991716,1.000000,0.989720,0.992138,0.943475,0.920948
INSTRUCTION_EXPENDITURE,0.045707,0.966589,0.989720,1.000000,0.972288,0.898873,0.877377
SUPPORT_SERVICES_EXPENDITURE,0.030917,0.987655,0.992138,0.972288,1.000000,0.953311,0.904419
OTHER_EXPENDITURE,-0.031047,0.971726,0.943475,0.898873,0.953311,1.000000,0.916780
CAPITAL_OUTLAY_EXPENDITURE,-0.054539,0.952952,0.920948,0.877377,0.904419,0.916780,1.000000


**(5)** When you need to choose the most appropriate variables for your model, would you prefer the first basic variables instead of the expenditure items? Why?

Instruction expenditure variable is more correlated with the overall score than the first principal component so I prefer to use instruction expenditure.